In [2]:
import glob
import geopandas as gpd
from shapely.ops import unary_union
import pandas as pd

In [3]:
years = ["2019","2020","2021" "2022"]
months = ["05","06","07","08"]
year = years[0]
resolution =  "res14"

In [4]:
def get_nieuwkoopseplassen_data(year):
        return glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[0]))\
        +glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[1]))\
        +glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[2]))\
        +glob.glob("E:/data/nieuwkoopse_plassen/yearmonth*.tif".replace("year", year).replace("month", months[3]))

In [5]:
def merge_hexagon_for_year(ayear):
        year_geojsons =glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+".geojson".replace("year", ayear).replace("month", months[0]))\
        +glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+".geojson".replace("year", ayear).replace("month", months[1]))\
        +glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+".geojson".replace("year", ayear).replace("month", months[2]))\
        +glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*"+resolution+".geojson".replace("year", ayear).replace("month", months[3]))

        print(len(year_geojsons))
        gdf = []
        for a_year_geojson in year_geojsons:
            agdf = gpd.read_file(a_year_geojson.replace("\\","/"))
            agdf['date'] = a_year_geojson.replace("\\","/").split("/")[-1].split("_")[0]+"-"+a_year_geojson.replace("\\","/").split("/")[-1].split("_")[1]
            gdf.append(agdf)

        gdf =pd.concat(gdf)

        source_gdf = gpd.read_file("C:/repos/satellite-images-nso/input_data/Nieuwkoopse Plassen & De Haeck.geojson")

        unary_union_geometry = unary_union(gdf['geometry'])


        print("Area covered with merge:")
        print(unary_union_geometry.intersection(source_gdf['geometry']).area/source_gdf['geometry'].area[0])
        print(gdf['date'].value_counts())
    
        return gdf

In [6]:
[item for sublist in [ get_nieuwkoopseplassen_data(ayear) for ayear in years ] for item in sublist] 

['E:/data/nieuwkoopse_plassen\\20190629_113712_SV1-02_50cm_RD_11bit_RGBI_Mijdrecht_Nieuwkoopse_Plassen_De_Haeck_cropped_ndwi_ndvi.tif',
 'E:/data/nieuwkoopse_plassen\\20190629_113714_SV1-02_50cm_RD_11bit_RGBI_Woerden_Nieuwkoopse_Plassen_De_Haeck_cropped_ndwi_ndvi.tif',
 'E:/data/nieuwkoopse_plassen\\20190826_111247_SV1-01_50cm_RD_11bit_RGBI_Mijdrecht_Nieuwkoopse_Plassen_De_Haeck_cropped_ndwi_ndvi.tif',
 'E:/data/nieuwkoopse_plassen\\20200713_111443_SV1-04_SV_RD_11bit_RGBI_50cm_Nieuwveen_Nieuwkoopse Plassen & De Haeck_cropped_ndwi_re_ndvi.tif',
 'E:/data/nieuwkoopse_plassen\\20200713_111445_SV1-04_SV_RD_11bit_RGBI_50cm_Boskoop_Nieuwkoopse Plassen & De Haeck_cropped_ndwi_re_ndvi.tif',
 'E:/data/nieuwkoopse_plassen\\20200812_111845_SV1-02_SV_RD_11bit_RGBI_50cm_VinkeveensePlassen_Nieuwkoopse Plassen & De Haeck_cropped_ndwi_re_ndvi.tif']

In [12]:
for year in years:
    print(year)
    try:
        year_geojsons =glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*res12.geojson".replace("year", year).replace("month", months[0]))\
        +glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*res12.geojson".replace("year", year).replace("month", months[1]))\
        +glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*res12.geojson".replace("year", year).replace("month", months[2]))\
        +glob.glob("E:/output/nieuwkoopse_plassen/yearmonth*res12.geojson".replace("year", year).replace("month", months[3]))

        print(len(year_geojsons))
        gdf = []
        for a_year_geojson in year_geojsons:
            agdf = gpd.read_file(a_year_geojson.replace("\\","/"))
            agdf['date'] = a_year_geojson.replace("\\","/").split("/")[-1].split("_")[0]+"-"+a_year_geojson.replace("\\","/").split("/")[-1].split("_")[1]
            gdf.append(agdf)

        gdf =pd.concat(gdf)

        source_gdf = gpd.read_file("C:/repos/satellite-images-nso/input_data/Nieuwkoopse Plassen & De Haeck.geojson")

        unary_union_geometry = unary_union(gdf['geometry'])


        print("Area covered with merge:")
        print(unary_union_geometry.intersection(source_gdf['geometry']).area/source_gdf['geometry'].area[0])
        print(gdf['date'].value_counts())
    except Exception as e:
        print(e)

2021
3


KeyboardInterrupt: 

In [7]:
def prioritize_high_with_date(df):
    # Find the index of 'High' value, if exists, and return the corresponding 'Date'
    high_index = df[df['label'] == "Waterplants"]
    if len(high_index) > 0:
        return high_index.iloc[0]
    else:
        # If 'High' is not found, return the first row
        return df.iloc[0]

In [8]:
for year in years:
    gdf = merge_hexagon_for_year(year)
    export_gdf = gdf.groupby(["hexagon_id"]).apply(prioritize_high_with_date).drop(['hexagon_id'], axis=1).reset_index()
    export_gdf.to_file('E:/output/nieuwkoopse_plassen/'+str(year)+'_bloom_period_merge_ndwi_ndvi_hexagons_'+resolution+'.geojson')

0


ValueError: No objects to concatenate